# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,632.356050,-0.452513,1234,631.903537,632.808562,1263.807074,1265.617124
6,595.161580,-0.374650,1234,594.786930,595.536230,1189.573860,1191.072459
7,537.471057,0.865384,1234,538.336440,536.605673,1076.672881,1073.211346
8,812.514728,0.736576,1234,813.251304,811.778152,1626.502607,1623.556304
9,950.606521,0.071704,1234,950.678226,950.534817,1901.356451,1901.069634
10,876.118527,-0.190163,1234,875.928365,876.308690,1751.856729,1752.617380
11,481.712522,-0.608907,1234,481.103615,482.321430,962.207230,964.642860
12,854.526541,0.463150,1234,854.989690,854.063391,1709.979380,1708.126782
13,998.964399,-0.561023,1234,998.403376,999.525422,1996.806751,1999.050845
14,438.898634,-0.690196,1234,438.208438,439.588830,876.416876,879.177659


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-631.903537,632.808562
6,-594.786930,595.536230
7,-538.336440,536.605673
8,-813.251304,811.778152
9,-950.678226,950.534817
10,-875.928365,876.308690
11,-481.103615,482.321430
12,-854.989690,854.063391
13,-998.403376,999.525422
14,-438.208438,439.588830


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-631.903537,632.808562
6,-594.786930,595.536230
7,-538.336440,536.605673
8,-813.251304,811.778152
9,-950.678226,950.534817
10,-875.928365,876.308690
11,-481.103615,482.321430
12,-854.989690,854.063391
13,-998.403376,999.525422
14,-438.208438,439.588830


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-631.90353714,  632.80856224],
       [-594.78692977,  595.53622955],
       [-538.3364405 ,  536.60567294],
       [-813.2513035 ,  811.77815187],
       [-950.67822558,  950.53481695],
       [-875.92836457,  876.30869001],
       [-481.1036151 ,  482.32142982],
       [-854.98969024,  854.06339108],
       [-998.40337571,  999.52542231],
       [-438.20843805,  439.5888296 ]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1261.807074,-1265.617124
6,1187.573860,-1191.072459
7,1074.672881,-1073.211346
8,1624.502607,-1623.556304
9,1899.356451,-1901.069634
10,1749.856729,-1752.617380
11,960.207230,-964.642860
12,1707.979380,-1708.126782
13,1994.806751,-1999.050845
14,874.416876,-879.177659


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.632356,0.273744
6,0.595162,0.312675
7,0.537471,0.932692
8,0.812515,0.868288
9,0.950607,0.535852
10,0.876119,0.404919
11,0.481713,0.195546
12,0.854527,0.731575
13,0.998964,0.219488
14,0.438899,0.154902


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,632.356050,-0.452513
6,595.161580,-0.374650
7,537.471057,0.865384
8,812.514728,0.736576
9,950.606521,0.071704
10,876.118527,-0.190163
11,481.712522,-0.608907
12,854.526541,0.463150
13,998.964399,-0.561023
14,438.898634,-0.690196


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
